# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
# Importar librerías

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [22]:
# Removing rows with missing data

spaceship_without_null = spaceship.dropna()

missing_values = spaceship_without_null.isnull().sum()
missing_values

# Extraer el primer carácter de la columna 'Cabin' usando .loc
spaceship_without_null.loc[:, 'Cabin'] = spaceship_without_null['Cabin'].str[0]

# Drop PassengerId and Name
spaceship_without_null = spaceship_without_null.drop(columns=['PassengerId', 'Name'])
spaceship_without_null

# Convertir 'Transported' a numérico usando .loc
spaceship_without_null.loc[:, 'Transported'] = spaceship_without_null['Transported'].astype(int)

# For non-numerical columns, do dummies.
spaceship_with_dummies = pd.get_dummies(spaceship_without_null, columns=['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP'])
spaceship_with_dummies



/Users/juanfransf/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
1,24.0,109.0,9.0,25.0,549.0,44.0,1,1,0,0,...,0,0,1,0,0,0,0,1,1,0
2,58.0,43.0,3576.0,0.0,6715.0,49.0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
3,33.0,0.0,1283.0,371.0,3329.0,193.0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
4,16.0,303.0,70.0,151.0,565.0,2.0,1,1,0,0,...,0,0,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,41.0,0.0,6819.0,0.0,1643.0,74.0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
8689,18.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,...,0,0,0,1,0,0,1,0,1,0
8690,26.0,0.0,0.0,1872.0,1.0,0.0,1,1,0,0,...,0,0,0,1,0,0,0,1,1,0
8691,32.0,0.0,1049.0,0.0,353.0,3235.0,0,0,1,0,...,0,1,0,0,0,1,0,0,1,0


In [23]:
# Separar las características (features) y la variable objetivo (target)
X = spaceship_with_dummies.drop(columns=['Transported'])
y = spaceship_with_dummies['Transported']

In [24]:
# Corregir valores faltantes con SimpleImputer
imputer = SimpleImputer(strategy='mean')
features = imputer.fit_transform(features)

**Perform Train Test Split**

In [25]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
# Escalar características
normalizer = MinMaxScaler()

normalizer.fit(X_train)

MinMaxScaler()

In [27]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [28]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)
bagging_reg

BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

In [29]:
# Entrenar Bagging model
bagging_reg.fit(X_train_norm, y_train)

BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

In [30]:
# Evaluación Bagging model

pred = bagging_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test_norm, y_test))

MAE 0.2696478896420208
RMSE 0.3666928624911862
R2 score 0.4620665821365939


- Random Forests

In [31]:
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=20)
forest

RandomForestRegressor(max_depth=20)

In [32]:
# Entrenar el modelo

forest.fit(X_train_norm, y_train)

RandomForestRegressor(max_depth=20)

In [33]:
# Evaluar el modelo

pred = forest.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", forest.score(X_test, y_test))

MAE 0.33653117565079826
RMSE 0.43714400714778073
R2 score 0.23550848567119342


- Gradient Boosting

In [34]:
# Initialize a AdaBoost model

gb_reg = GradientBoostingRegressor(max_depth=20,
                                   n_estimators=100)



In [35]:
# Entrenar el modelo

gb_reg.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=20)

In [36]:
# Evaluar modelo

pred = gb_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", gb_reg.score(X_test_norm, y_test))

MAE 0.602600521865875
RMSE 0.7386853274281514
R2 score -1.182943808499925


- Adaptive Boosting

In [37]:
# Initialize a AdaBoost model
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)

# Training the model

ada_reg.fit(X_train_norm, y_train)

# Evaluate the model

pred = ada_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", ada_reg.score(X_test_norm, y_test))

MAE 0.2286152102900989
RMSE 0.41137983650726057
R2 score 0.32296734900976143


Which model is the best and why?

Si el objetivo principal es minimizar el error absoluto promedio, Adaptive Boosting sería la mejor opción.
Si es importante capturar mejor la variabilidad de los datos y penalizar los errores grandes, Bagging and Pasting sería la mejor opción

En general, Bagging and Pasting parece ser el modelo más equilibrado considerando todas las métricas, especialmente porque tiene el mejor RMSE y R2 Score.